In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind
import geocoder
import folium
from haversine import haversine
% matplotlib inline
import seaborn as sns
from matplotlib import rcParams

In [2]:
# Get traffic data csv 
# Drop everything but segment ID and addresses (main and 2 ending streets), pair of intersections 
# Convert to coordinates and save to file to make a dictionary to basically apply it to the old csv 

df_traffic_data = pd.read_csv('Traffic_Volume_Counts__2012-2013_.csv')
df_traffic_data = df_traffic_data.dropna()

In [3]:
# New dataframe with only the specific columns and clean this data
# Drop duplicates from the dataframe 

df_traffic_cleaned = df_traffic_data[['Segment ID', 'Roadway Name', 'From', 'To']].copy()
df_traffic_cleaned = df_traffic_cleaned.drop_duplicates()

# df_traffic_cleaned = df_traffic_cleaned.reset_index(drop=True)

# rename columns df.columns = ['a', 'b']

# unique_segments = len(set(df_traffic_cleaned['Segment ID']))

In [4]:
# Generate empty columns for GPS coords 

df_traffic_cleaned['From Lat'] = np.nan
df_traffic_cleaned['From Lon'] = np.nan
df_traffic_cleaned['To Lat'] = np.nan
df_traffic_cleaned['To Lon'] = np.nan

In [5]:
#rcParams['figure.figsize'] = 8, 6
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Tahoma']
#sns.set(style='whitegrid', context ='paper')

In [6]:
# Generate gps coordinates for the From intersection 

for i in df_traffic_cleaned.index:
    # Concatenate the street names as an intersection to find the coords
    road = (df_traffic_cleaned.ix[i,'Roadway Name']) + ' & ' + (df_traffic_cleaned.ix[i,'From']) + ', NY'
    
    # Send the address 
    # If the address isn't found then the coordinates column remains as NaN 
    g = geocoder.google(road)
    df_traffic_cleaned.set_value(i, 'From Lat', g.lat)
    df_traffic_cleaned.set_value(i, 'From Lon', g.lng)

In [7]:
# Generate gps coordinates for the To intersection 

for i in df_traffic_cleaned.index:
    # Concatenate the street names as an intersection to find the coords
    road = (df_traffic_cleaned.ix[i,'Roadway Name']) + ' & ' + (df_traffic_cleaned.ix[i,'To']) + ', NY'
    
    # Send the address 
    # If the address isn't found then the coordinates column remains as NaN 
    g = geocoder.google(road)
    df_traffic_cleaned.set_value(i, 'To Lat', g.lat)
    df_traffic_cleaned.set_value(i, 'To Lon', g.lng)

In [12]:
# Drop all the columns where the geocoding failed to obtain coordinates 
# Reindex 

df_traffic_cleaned  = df_traffic_cleaned.dropna()
df_traffic_cleaned = df_traffic_cleaned.reset_index(drop=True)

In [14]:
df_traffic_cleaned

Segment ID              Roadway Name                  From  \
0          2153              HUGUENOT AVE            WOODROW RD   
1          6644        RICHMOND HILL ROAD         EDINBORO ROAD   
2          7564             MORLEY AVENUE        DALTON  AVENUE   
3         11599             OCEAN TERRACE    BUTTERWORTH AVENUE   
4         12945           LITTLE CLOVE RD         CAYUGA AVENUE   
5         13133                  CLOVE RD         HOWARD AVENUE   
6         14602           HYLAN BOULEVARD      FINGERBOARD ROAD   
7         17319                 86 STREET              7 AVENUE   
8         17652                  4 AVENUE             68 STREET   
9         18635              CROPSEY AVE            BAY 22nd ST   
10        18641                 BATH AVE            BAY 22nd ST   
11        18653               BENSON AVE               19th AVE   
12        18846                SHORE PKWY           BAY 46th ST   
13        19202                  86th ST              W 11th ST   
14        19244                 KINGS HWY             W 11th ST   
15        19313                     AVE U             W 10th ST   
16        19437            BAY RIDGE PKWY              11TH AVE   
17        19463                  8 AVENUE             66 STREET   
18        19566                 65 STREET             10 AVENUE   
19        19641                 13 AVENUE             72 STREET   
20        19761                 14 AVENUE             72 STREET   
21        19805                 60 STREET             11 AVENUE   
22        19813                   86th ST              19th AVE   
23        19991                 18 AVENUE             72 STREET   
24        20157                 65 STREET             18 AVENUE   
25        20278                 60 STREET             18 AVENUE   
26        20335                     AVE P             W 10th ST   
27        20340                  BAY PKWY               72nd ST   
28        20561                  7 AVENUE             65 STREET   
29        21028                  3 AVENUE             29 STREET   
..          ...                       ...                   ...   
337      158683       EAST HOUSTON STREET              BROADWAY   
338      158764                   7 AVE            WEST 122 ST    
339      158858                  BROADWAY        WEST 60 STREET   
340      158964                 LENOX AVE           WEST 123 ST   
341      158972                 LENOX AVE           WEST 111 ST   
342      159002                   7 AVE            WEST 117 ST    
343      159082                BROADWAY            WEST 98 ST     
344      159183                  BROADWAY        WEST 74 STREET   
345      159216               PARK AVENUE        EAST 59 STREET   
346      164301            ALLEN STREET        DELANCEY STREET    
347      164329            EAST 18 STREET              BROADWAY   
348      164341            NORFOLK STREET       DELANCEY STREET   
349      165258                MEEKER AVE            LEONARD ST   
350      167027             KINGS HIGHWAY              AVENUE I   
351      167454           ROCKAWAY AVENUE        HEGEMAN AVENUE   
352      174149         EAST 161 STREET        GRAND CONCOURSE    
353      174218         EAST 165 STREET         CARROLL PLACE     
354      174566         HENRY HUDSON PKWY         FAIRFIELD AVE   
355      176380        EAST 204 STREET       GRAND CONCOURSE      
356      176399       SOUTHERN BOULEVARD       CROTONA PARKWAY    
357      176426           CROTONA AVENUE     EAST FORDHAM ROAD    
358      178157            ZEREGA AVENUE    BRUCKNER BOULEVARD    
359      184562     GRAND CONCOURSE_MAIN      EAST 172 STREET     
360      192292                2 AVENUE        EAST 59 STREET     
361      154496   CROSS BRONX EXPRESSWAY        MORRIS AVENUE     
362       72331                  E 174 ST       MORRIS AVENUE     
363      154454   CROSS BRONX EXPRESSWAY        MARMION AVENUE    
364       78800                  E 176 ST       MARM

In [16]:
df_traffic_cleaned.to_csv('traffic_count_coordinates.csv')